In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
칼럼 =['연번','자치구','담당','구분(의장,부의장,상임위원장)','집행일시','시분값', \
       '집행장소','집행주소','결재방법','집행금액','집행목적/집행내역/내용','대상인원','50만원이상증빙서류','비고']

In [3]:
기본경로 = r'C:\Users\user\1코드들\2018년_정공센_알권리감시단_업무추진비_코드\Files\공개자료_업데이트\\'